In [1]:
import warnings
warnings.simplefilter('ignore')

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn import preprocessing
from sklearn.preprocessing import normalize
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import VotingClassifier
#Stop warnings
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

C:\Users\Rowena\Anaconda3\envs\python3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [43]:
from sklearn.metrics import precision_recall_fscore_support

In [3]:
data = pd.read_csv('Pokemon.csv')

In [4]:
data.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [5]:
data = data.set_index('#')
data.head()
#Type1 is our response variable we wish to classify
type1 = data.pop('Type 1')

In [6]:
data.head()

,Name,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
#,,,,,,,,,,,
1,Bulbasaur,Poison,318,45,49,49,65,65,45,1,False
2,Ivysaur,Poison,405,60,62,63,80,80,60,1,False
3,Venusaur,Poison,525,80,82,83,100,100,80,1,False
3,VenusaurMega Venusaur,Poison,625,80,100,123,122,120,80,1,False
4,Charmander,NaN,309,39,52,43,60,50,65,1,False


# Data Processing

In [7]:
type1.value_counts()

Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Rock         44
Electric     44
Ghost        32
Ground       32
Dragon       32
Dark         31
Poison       28
Fighting     27
Steel        27
Ice          24
Fairy        17
Flying        4
Name: Type 1, dtype: int64

In [8]:
# create a binary response variable for each class
Y = pd.get_dummies(type1)
np.sum(Y, axis=0)

Bug          69
Dark         31
Dragon       32
Electric     44
Fairy        17
Fighting     27
Fire         52
Flying        4
Ghost        32
Grass        70
Ground       32
Ice          24
Normal       98
Poison       28
Psychic      57
Rock         44
Steel        27
Water       112
dtype: int64

In [9]:
#What response variable currently looks like
Y.head()

,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
#,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [10]:
dummies2list = []
for x in Y:
    word = 'Type_1_' + x
    dummies2list.append(word)
Y.columns = dummies2list

In [11]:
#Allows us to distinguish between type 1 and type 2 later on
Y.head()

,Type_1_Bug,Type_1_Dark,Type_1_Dragon,Type_1_Electric,Type_1_Fairy,Type_1_Fighting,Type_1_Fire,Type_1_Flying,Type_1_Ghost,Type_1_Grass,Type_1_Ground,Type_1_Ice,Type_1_Normal,Type_1_Poison,Type_1_Psychic,Type_1_Rock,Type_1_Steel,Type_1_Water
#,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [12]:
#Processes the generation column and type2 column by converting them into binary variables
dummies = pd.get_dummies(data['Generation'])
dummies3 = pd.get_dummies(data['Type 2'])

dummies1list = []
for x in dummies:
    word = 'Generation_' + str(x)
    dummies1list.append(word)
    
dummies.columns = dummies1list
step_1 = pd.concat([data, dummies], axis=1)  

dummies3list = []
for x in dummies3:
    word = 'Type_2_' + x
    dummies3list.append(word)
dummies3.columns = dummies3list
final_step = pd.concat([step_1,dummies3], axis = 1)

del final_step['Type 2']
del final_step['Generation']
del final_step['Name']
final_step

#Converts the boolean column into a column of 0's and 1's
final_step.Legendary = final_step.Legendary.astype(int)

In [13]:
final_step.head()

,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary,Generation_1,Generation_2,...,Type_2_Ghost,Type_2_Grass,Type_2_Ground,Type_2_Ice,Type_2_Normal,Type_2_Poison,Type_2_Psychic,Type_2_Rock,Type_2_Steel,Type_2_Water
#,,,,,,,,,,,,,,,,,,,,,
1,318,45,49,49,65,65,45,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,405,60,62,63,80,80,60,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,525,80,82,83,100,100,80,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,625,80,100,123,122,120,80,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,309,39,52,43,60,50,65,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
final_step.columns

Index(['Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed',
       'Legendary', 'Generation_1', 'Generation_2', 'Generation_3',
       'Generation_4', 'Generation_5', 'Generation_6', 'Type_2_Bug',
       'Type_2_Dark', 'Type_2_Dragon', 'Type_2_Electric', 'Type_2_Fairy',
       'Type_2_Fighting', 'Type_2_Fire', 'Type_2_Flying', 'Type_2_Ghost',
       'Type_2_Grass', 'Type_2_Ground', 'Type_2_Ice', 'Type_2_Normal',
       'Type_2_Poison', 'Type_2_Psychic', 'Type_2_Rock', 'Type_2_Steel',
       'Type_2_Water'],
      dtype='object')

# Modelling

In [15]:
#Creating training and validation set
X_train, X_test, y_train, y_test = train_test_split( final_step, Y, test_size = 0.3, random_state = 100)

#Normalise the features for some models
norm_train = preprocessing.normalize(X_train)
norm_test = preprocessing.normalize(X_test)

In [16]:
#Computes accuracy and confusion matrices
#Not normalising features
def predictingModel(model):
    
    accuracy = []
    confusion_matrices = []
    
    for y_type in y_train:
        model.fit(X_train, (y_train[y_type]))
        pred = model.predict(X_test)
        confusion_matrices.append(confusion_matrix(y_test[y_type],pred))
        accuracy.append(accuracy_score(y_test[y_type],pred))
    return accuracy, confusion_matrices

In [17]:
#For models where we need to normalise features
def predictingModel2(model):
    
    accuracy = []
    confusion_matrices = []
    
    for y_type in y_train:
        model.fit(norm_train, (y_train[y_type]))
        pred = model.predict(norm_test)
        confusion_matrices.append(confusion_matrix(y_test[y_type],pred))
        accuracy.append(accuracy_score(y_test[y_type],pred))
    return accuracy, confusion_matrices

In [83]:
def metrics(model):
    
    precision = []
    recall = []
    fscore = []
    support = []

    for y_type in y_train:
        model.fit(norm_train, (y_train[y_type]))
        pred = model.predict(norm_test)
        p, r, f, s = precision_recall_fscore_support(y_test[y_type],pred,average='binary')
        precision.append(p)
        recall.append(r)
        fscore.append(f)
        support.append(s)
    
    return precision, recall, fscore, support

#### Models to be used: Naive Bayes, Linear Discriminant Analysis, Quadratic Discriminant Analysis, Logistic Regression, Decision Tree, Random Forest, Extremely Random Forest, Adaptive Boost, Gradient Boosting

##### Naive Bayes

In [18]:
#Naive Bayes
NaiveB = GaussianNB()
NB_A, NB_C = predictingModel2(NaiveB)

##### Linear Discriminant Analysis

In [19]:
#Linear Discriminant analysis
LDA = LinearDiscriminantAnalysis()
LDA_A, LDA_C = predictingModel2(LDA)

##### Quadratic Discriminant Analysis

In [20]:
#Quadratic Discriminant analysis
QDA = QuadraticDiscriminantAnalysis()
QDA_A, QDA_C = predictingModel2(QDA)

##### Logistic Regression

In [21]:
#Logistic regression
LogReg = LogisticRegression(random_state=450411920)
LG_A, LG_C = predictingModel(LogReg)

##### Decision Tree

For the subsequent models, we have hyperparameters to tune. Therefore we use randomized search instead of grid search in order to save on computation time. This tries out random hyperparameters instead of all possible combination of hyperparameters, dramatically cutting down computation.

In [22]:
# Setup the parameters and distributions to sample from: param_dist
param_dist = {"max_depth": [5, None],
              "max_features": range(1, 9),
              "min_samples_leaf": range(1, 9),
              "criterion": ["gini", "entropy"]}

# Instantiate a Decision Tree classifier: tree
tree = DecisionTreeClassifier()

# Instantiate the RandomizedSearchCV object: tree_cv
tree_cv = RandomizedSearchCV(tree,param_dist, cv=10, scoring='accuracy', n_iter=10, random_state=5)

In [23]:
DT_A, DT_C = predictingModel(tree_cv)

##### Random Forest

In [24]:
# Setup the parameters and distributions to sample from: param_dist
param_dist = {"n_estimators": range(5,15),
              "max_depth": [3, None],
              "max_features": range(1, 9),
              "min_samples_split": range(2,5),
              "min_samples_leaf": range(1, 9),
              }

# Instantiate a Extremely Random Forest classifier: randext
randFor = RandomForestClassifier()

# Instantiate the RandomizedSearchCV object: randFor_cv
randFor_cv = RandomizedSearchCV(randFor,param_dist, cv=10, scoring='accuracy', n_iter=10, random_state=5)

In [25]:
RF_A, RF_C = predictingModel(randFor_cv)

##### Extremely Random Forest

In [26]:
# Setup the parameters and distributions to sample from: param_dist
param_dist = {"n_estimators": range(5,15),
              "max_depth": [3, None],
              "max_features": range(1, 9),
              "min_samples_split": range(2,5),
              "min_samples_leaf": range(1, 9),
              }

# Instantiate a Extremely Random Forest classifier: randext
randExt = ExtraTreesClassifier()

# Instantiate the RandomizedSearchCV object: randext_cv
randExt_cv = RandomizedSearchCV(randExt,param_dist, cv=10, scoring='accuracy', n_iter=10, random_state=5)

In [27]:
ERF_A, ERF_C = predictingModel(randExt_cv)

##### Adaptive Boosting

In [28]:
# Setup the parameters and distributions to sample from: param_dist
param_dist = {"n_estimators": range(330,400,10),
              "learning_rate": range(1,5),
              }

# Instantiate a Adaptie boosting classifier: tree
randAda = AdaBoostClassifier()

# Instantiate the RandomizedSearchCV object: randada_cv
randAda_cv = RandomizedSearchCV(randAda,param_dist, cv=10, scoring='accuracy', n_iter=10, random_state=5)

In [29]:
AB_A, AB_C = predictingModel(randAda_cv)

##### Gradient Boosting

In [30]:
# Setup the parameters and distributions to sample from: param_dist
param_dist = {"loss": ["deviance", "exponential"],
              "n_estimators": range(250,550,50),
              "learning_rate": [0.1],
              "min_samples_split" : [2,4],
              "min_samples_leaf" : range(1,3)
              }

# Instantiate a GB boosting classifier: randGB
randGB = GradientBoostingClassifier()

# Instantiate the RandomizedSearchCV object: randGB_cv
randGB_cv = RandomizedSearchCV(randGB,param_dist, cv=10, scoring='accuracy', n_iter=10, random_state=5)

In [31]:
GB_A, GB_C = predictingModel(randGB_cv)

Looking at how our models performed so far:

In [32]:
models = ['Naive Bayes', 'LDA', 'QDA', 'Logistic Regression','Decision Tree','Random Forest','Extremely Random Forest','Adaptive Boosting','Gradient Boosting']

In [33]:
accuracy_scores = [NB_A, LDA_A, QDA_A, LG_A,DT_A, RF_A, ERF_A, AB_A, GB_A]

In [34]:
accuracy = pd.DataFrame(data = accuracy_scores).transpose()
accuracy.columns = models
accuracy.set_index(Y.columns).round(3)

,Naive Bayes,LDA,QDA,Logistic Regression,Decision Tree,Random Forest,Extremely Random Forest,Adaptive Boosting,Gradient Boosting
Type_1_Bug,0.325,0.917,0.329,0.938,0.942,0.942,0.921,0.917,0.921
Type_1_Dark,0.492,0.925,0.950,0.950,0.950,0.950,0.950,0.933,0.946
Type_1_Dragon,0.575,0.871,0.954,0.954,0.925,0.950,0.954,0.938,0.946
Type_1_Electric,0.333,0.925,0.354,0.929,0.929,0.938,0.929,0.921,0.946
Type_1_Fairy,0.846,0.962,0.967,0.967,0.967,0.967,0.967,0.958,0.962
Type_1_Fighting,0.383,0.954,0.958,0.962,0.971,0.971,0.971,0.929,0.958
Type_1_Fire,0.233,0.946,0.233,0.946,0.946,0.946,0.946,0.912,0.929
Type_1_Flying,0.942,0.975,0.992,0.992,0.992,0.992,0.992,0.992,0.988
Type_1_Ghost,0.312,0.967,0.496,0.979,0.983,0.983,0.983,0.975,0.979
Type_1_Grass,0.233,0.912,0.212,0.933,0.929,0.921,0.933,0.896,0.929


In [35]:
accuracy.describe().round(3)

,Naive Bayes,LDA,QDA,Logistic Regression,Decision Tree,Random Forest,Extremely Random Forest,Adaptive Boosting,Gradient Boosting
count,18.000,18.000,18.000,18.000,18.000,18.000,18.000,18.000,18.000
mean,0.426,0.927,0.670,0.942,0.941,0.944,0.944,0.923,0.936
std,0.197,0.035,0.331,0.035,0.036,0.034,0.033,0.045,0.038
min,0.221,0.850,0.192,0.846,0.854,0.862,0.867,0.792,0.821
25%,0.314,0.918,0.335,0.933,0.929,0.934,0.930,0.906,0.929
50%,0.358,0.929,0.902,0.952,0.948,0.950,0.952,0.929,0.946
75%,0.483,0.949,0.957,0.961,0.965,0.965,0.965,0.949,0.950
max,0.942,0.975,0.992,0.992,0.992,0.992,0.992,0.992,0.988


In [36]:
for i,j in zip(models,accuracy_scores):
    print(i,' average accuracy is: ', np.average(j).round(3))

Naive Bayes  average accuracy is:  0.426
LDA  average accuracy is:  0.927
QDA  average accuracy is:  0.67
Logistic Regression  average accuracy is:  0.942
Decision Tree  average accuracy is:  0.941
Random Forest  average accuracy is:  0.944
Extremely Random Forest  average accuracy is:  0.944
Adaptive Boosting  average accuracy is:  0.923
Gradient Boosting  average accuracy is:  0.936


### Ensemble Creation

From this, we create a final ensemble by combining the top 3 uncorrelated models and take a soft vote (by assigning a higher weight to the vote for a "better" model) on which class does an observation belong to. The top uncorrelated models are:
- Extremely Random Forest

- Logistic Regression

- Gradient Boosting

In [37]:
finalensem = VotingClassifier(estimators=[('Ext Rand Forest', randExt_cv), ('Log Reg', LogReg), ('GradientBoost', randGB_cv)], voting='soft')

In [38]:
accuracy_finalensem = []
for y_type in y_train:
    finalensem.fit(X_train, y_train[y_type])
    pred = finalensem.predict(X_test)
    accuracy_finalensem.append(accuracy_score(y_test[y_type],pred))

In [145]:
confusion_finalensem = []
for y_type in y_train:
    finalensem.fit(X_train, y_train[y_type])
    pred = finalensem.predict(X_test)
    confusion_finalensem.append(confusion_matrix(y_test[y_type],pred))

In [39]:
models.append('Final Ensemble')
accuracy_scores.append(accuracy_finalensem)

In [40]:
accuracy2 = pd.DataFrame(data = accuracy_scores).transpose()
accuracy2.columns = models
accuracy2.set_index(Y.columns).round(3)

,Naive Bayes,LDA,QDA,Logistic Regression,Decision Tree,Random Forest,Extremely Random Forest,Adaptive Boosting,Gradient Boosting,Final Ensemble
Type_1_Bug,0.325,0.917,0.329,0.938,0.942,0.942,0.921,0.917,0.921,0.938
Type_1_Dark,0.492,0.925,0.950,0.950,0.950,0.950,0.950,0.933,0.946,0.950
Type_1_Dragon,0.575,0.871,0.954,0.954,0.925,0.950,0.954,0.938,0.946,0.954
Type_1_Electric,0.333,0.925,0.354,0.929,0.929,0.938,0.929,0.921,0.946,0.929
Type_1_Fairy,0.846,0.962,0.967,0.967,0.967,0.967,0.967,0.958,0.962,0.967
Type_1_Fighting,0.383,0.954,0.958,0.962,0.971,0.971,0.971,0.929,0.958,0.971
Type_1_Fire,0.233,0.946,0.233,0.946,0.946,0.946,0.946,0.912,0.929,0.946
Type_1_Flying,0.942,0.975,0.992,0.992,0.992,0.992,0.992,0.992,0.988,0.992
Type_1_Ghost,0.312,0.967,0.496,0.979,0.983,0.983,0.983,0.975,0.979,0.983
Type_1_Grass,0.233,0.912,0.212,0.933,0.929,0.921,0.933,0.896,0.929,0.933


In [41]:
accuracy2.describe().round(3)

,Naive Bayes,LDA,QDA,Logistic Regression,Decision Tree,Random Forest,Extremely Random Forest,Adaptive Boosting,Gradient Boosting,Final Ensemble
count,18.000,18.000,18.000,18.000,18.000,18.000,18.000,18.000,18.000,18.000
mean,0.426,0.927,0.670,0.942,0.941,0.944,0.944,0.923,0.936,0.944
std,0.197,0.035,0.331,0.035,0.036,0.034,0.033,0.045,0.038,0.033
min,0.221,0.850,0.192,0.846,0.854,0.862,0.867,0.792,0.821,0.862
25%,0.314,0.918,0.335,0.933,0.929,0.934,0.930,0.906,0.929,0.934
50%,0.358,0.929,0.902,0.952,0.948,0.950,0.952,0.929,0.946,0.952
75%,0.483,0.949,0.957,0.961,0.965,0.965,0.965,0.949,0.950,0.962
max,0.942,0.975,0.992,0.992,0.992,0.992,0.992,0.992,0.988,0.992


Conclusively, we have an average accuracy of 94.5% for our final ensemble, an improvement from the 3 base models we included in our ensemble.

## Model evaluation

In [45]:
models_estimated = [NaiveB, LDA, QDA, LogReg, tree_cv, randFor_cv, randExt_cv, randAda_cv, randGB_cv, finalensem]

In [84]:
precision_all = []
recall_all = []
fscore_all = []
support_all = []

for model_estimate in models_estimated:
    metrics_model = metrics(model_estimate)
    precision_all.append(metrics_model[0])
    recall_all.append(metrics_model[1])
    fscore_all.append(metrics_model[2])
    support_all.append(metrics_model[3])

In [93]:
print(len(y_train))
print(len(y_test))

560
240


In [117]:
total_proportion = 0
weight = []
for type1_test in y_test.columns:
    weight.append(sum(y_test[type1_test])/len(y_test))
    total_proportion += sum(y_test[type1_test])/len(y_test)
    print(type1_test + ': ' + str(sum(y_test[type1_test])) + '; Proportion =', np.round(sum(y_test[type1_test])/len(y_test),3))
print('Sum of proportions:', total_proportion) # should equal 1
print('Average proportion:', np.round(total_proportion/18,3))

Type_1_Bug: 14; Proportion = 0.058
Type_1_Dark: 12; Proportion = 0.05
Type_1_Dragon: 11; Proportion = 0.046
Type_1_Electric: 17; Proportion = 0.071
Type_1_Fairy: 8; Proportion = 0.033
Type_1_Fighting: 7; Proportion = 0.029
Type_1_Fire: 13; Proportion = 0.054
Type_1_Flying: 2; Proportion = 0.008
Type_1_Ghost: 5; Proportion = 0.021
Type_1_Grass: 16; Proportion = 0.067
Type_1_Ground: 11; Proportion = 0.046
Type_1_Ice: 11; Proportion = 0.046
Type_1_Normal: 30; Proportion = 0.125
Type_1_Poison: 8; Proportion = 0.033
Type_1_Psychic: 17; Proportion = 0.071
Type_1_Rock: 17; Proportion = 0.071
Type_1_Steel: 10; Proportion = 0.042
Type_1_Water: 31; Proportion = 0.129
Sum of proportions: 1.0
Average proportion: 0.056


In [119]:
print(np.round(weight,3))
print(np.sum(weight))

[ 0.058  0.05   0.046  0.071  0.033  0.029  0.054  0.008  0.021  0.067
  0.046  0.046  0.125  0.033  0.071  0.071  0.042  0.129]
1.0


In [146]:
c_matrices = [NB_C, LDA_C, QDA_C, LG_C, DT_C, RF_C, ERF_C, AB_C, GB_C, confusion_finalensem]

### Precision
The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.

In [147]:
precision2 = []
for model in c_matrices:
    precision_model = []
    for type1 in model:
        precision_model.append(type1[1][1] / (type1[1][1] + type1[0][1]))
    precision2.append(precision_model)

In [148]:
len(precision2)

10

In [158]:
precision = pd.DataFrame(data = precision2).transpose()
precision.columns = models
precision.set_index(Y.columns).round(3)

,Naive Bayes,LDA,QDA,Logistic Regression,Decision Tree,Random Forest,Extremely Random Forest,Adaptive Boosting,Gradient Boosting,Final Ensemble
Type_1_Bug,0.070,0.125,0.080,0.333,0.500,NaN,0.0,0.286,0.273,0.250
Type_1_Dark,0.063,0.200,NaN,NaN,NaN,NaN,NaN,0.000,0.000,NaN
Type_1_Dragon,0.058,0.083,NaN,NaN,0.182,0.00,0.5,0.167,0.250,0.500
Type_1_Electric,0.077,0.333,0.084,NaN,NaN,1.00,NaN,0.375,0.833,NaN
Type_1_Fairy,0.128,0.429,NaN,NaN,NaN,NaN,NaN,0.000,0.000,NaN
Type_1_Fighting,0.045,0.167,0.000,0.250,NaN,NaN,NaN,0.083,0.286,0.500
Type_1_Fire,0.062,0.500,0.062,NaN,NaN,NaN,NaN,0.214,0.167,NaN
Type_1_Flying,0.125,0.167,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN
Type_1_Ghost,0.024,0.200,0.032,NaN,1.000,1.00,1.0,0.333,0.500,1.000
Type_1_Grass,0.071,0.222,0.070,NaN,0.400,0.00,0.5,0.200,0.400,0.500


In [159]:
precision.describe().round(3)

,Naive Bayes,LDA,QDA,Logistic Regression,Decision Tree,Random Forest,Extremely Random Forest,Adaptive Boosting,Gradient Boosting,Final Ensemble
count,18.000,18.000,10.000,6.000,7.000,8.000,6.000,17.000,18.000,10.000
mean,0.077,0.244,0.097,0.386,0.353,0.469,0.500,0.204,0.280,0.520
std,0.038,0.144,0.071,0.342,0.329,0.471,0.447,0.179,0.239,0.304
min,0.024,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,0.058,0.167,0.064,0.236,0.174,0.000,0.125,0.083,0.031,0.406
50%,0.063,0.236,0.082,0.292,0.222,0.375,0.500,0.167,0.279,0.500
75%,0.090,0.333,0.127,0.458,0.450,1.000,0.875,0.286,0.446,0.554
max,0.160,0.500,0.250,1.000,1.000,1.000,1.000,0.667,0.833,1.000


### Recall
The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.

In [160]:
recall2 = []
for model in c_matrices:
    recall_model = []
    for type1 in model:
        recall_model.append(type1[1][1] / (type1[1][1] + type1[1][0]))
    recall2.append(recall_model)

In [161]:
recall = pd.DataFrame(data = recall2).transpose()
recall.columns = models
recall.set_index(Y.columns).round(3)

,Naive Bayes,LDA,QDA,Logistic Regression,Decision Tree,Random Forest,Extremely Random Forest,Adaptive Boosting,Gradient Boosting,Final Ensemble
Type_1_Bug,0.857,0.071,1.000,0.071,0.071,0.000,0.000,0.286,0.214,0.071
Type_1_Dark,0.667,0.167,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Type_1_Dragon,0.545,0.182,0.000,0.000,0.182,0.000,0.091,0.091,0.091,0.091
Type_1_Electric,0.765,0.059,0.824,0.000,0.000,0.118,0.000,0.176,0.294,0.000
Type_1_Fairy,0.625,0.375,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Type_1_Fighting,1.000,0.143,0.000,0.143,0.000,0.000,0.000,0.143,0.286,0.143
Type_1_Fire,0.923,0.077,0.923,0.000,0.000,0.000,0.000,0.231,0.077,0.000
Type_1_Flying,1.000,0.500,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Type_1_Ghost,0.800,0.200,0.800,0.000,0.200,0.200,0.200,0.200,0.200,0.200
Type_1_Grass,0.875,0.125,0.875,0.000,0.125,0.000,0.125,0.188,0.125,0.125


In [162]:
recall.describe().round(3)

,Naive Bayes,LDA,QDA,Logistic Regression,Decision Tree,Random Forest,Extremely Random Forest,Adaptive Boosting,Gradient Boosting,Final Ensemble
count,18.000,18.000,18.000,18.000,18.000,18.000,18.00,18.000,18.000,18.000
mean,0.804,0.169,0.434,0.026,0.039,0.026,0.03,0.129,0.142,0.066
std,0.136,0.126,0.458,0.046,0.066,0.054,0.06,0.097,0.117,0.077
min,0.545,0.000,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000
25%,0.711,0.080,0.000,0.000,0.000,0.000,0.00,0.023,0.016,0.000
50%,0.809,0.167,0.265,0.000,0.000,0.000,0.00,0.136,0.153,0.036
75%,0.874,0.195,0.873,0.044,0.065,0.024,0.00,0.200,0.229,0.113
max,1.000,0.500,1.000,0.143,0.200,0.200,0.20,0.286,0.353,0.235


### F-measure

In [163]:
fmeasure_all = []
for model in c_matrices:
    fmeasure_model = []
    for type1 in model:
        fmeasure_model.append(2*type1[1][1] / (2*type1[1][1] + type1[1][0] + type1[0][1]))
    fmeasure_all.append(fmeasure_model)

In [164]:
fmeasure = pd.DataFrame(data = fmeasure_all).transpose()
fmeasure.columns = models
fmeasure.set_index(Y.columns).round(3)

,Naive Bayes,LDA,QDA,Logistic Regression,Decision Tree,Random Forest,Extremely Random Forest,Adaptive Boosting,Gradient Boosting,Final Ensemble
Type_1_Bug,0.129,0.091,0.148,0.118,0.125,0.000,0.000,0.286,0.240,0.111
Type_1_Dark,0.116,0.182,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Type_1_Dragon,0.105,0.114,0.000,0.000,0.182,0.000,0.154,0.118,0.133,0.154
Type_1_Electric,0.140,0.100,0.153,0.000,0.000,0.211,0.000,0.240,0.435,0.000
Type_1_Fairy,0.213,0.400,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Type_1_Fighting,0.086,0.154,0.000,0.182,0.000,0.000,0.000,0.105,0.286,0.222
Type_1_Fire,0.115,0.133,0.115,0.000,0.000,0.000,0.000,0.222,0.105,0.000
Type_1_Flying,0.222,0.250,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Type_1_Ghost,0.046,0.200,0.062,0.000,0.333,0.333,0.333,0.250,0.286,0.333
Type_1_Grass,0.132,0.160,0.129,0.000,0.190,0.000,0.200,0.194,0.190,0.200


In [165]:
fmeasure.describe().round(3)

,Naive Bayes,LDA,QDA,Logistic Regression,Decision Tree,Random Forest,Extremely Random Forest,Adaptive Boosting,Gradient Boosting,Final Ensemble
count,18.000,18.000,18.000,18.000,18.000,18.000,18.000,18.000,18.000,18.000
mean,0.139,0.176,0.091,0.040,0.057,0.045,0.050,0.148,0.183,0.105
std,0.061,0.100,0.111,0.068,0.096,0.092,0.101,0.114,0.147,0.120
min,0.046,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,0.106,0.119,0.000,0.000,0.000,0.000,0.000,0.026,0.021,0.000
50%,0.116,0.171,0.031,0.000,0.000,0.000,0.000,0.138,0.213,0.056
75%,0.164,0.242,0.152,0.083,0.099,0.043,0.000,0.248,0.286,0.192
max,0.271,0.400,0.340,0.182,0.333,0.333,0.333,0.308,0.435,0.333


### Weighted accuracy

In [166]:
accuracy_w_all = []
TOTAL_TEST = len(y_test)

for model in c_matrices:
    accuracy_w_model = []
    for type1 in model:
        w1 = type1[1][1]/TOTAL_TEST
        w2 = type1[1][0]/TOTAL_TEST
        w3 = type1[0][1]/TOTAL_TEST
        w4 = type1[0][0]/TOTAL_TEST
        accuracy_w_model.append((w1*type1[1][1] + w4*type1[0][0]) / (w1*type1[1][1] + w2*type1[1][0] + w3*type1[0][1] + w4*type1[0][0]))
    accuracy_w_all.append(accuracy_w_model)

In [167]:
accuracy_w = pd.DataFrame(data = accuracy_w_all).transpose()
accuracy_w.columns = models
accuracy_w.set_index(Y.columns).round(3)

,Naive Bayes,LDA,QDA,Logistic Regression,Decision Tree,Random Forest,Extremely Random Forest,Adaptive Boosting,Gradient Boosting,Final Ensemble
Type_1_Bug,0.149,0.995,0.146,0.997,0.997,0.996,0.995,0.996,0.996,0.996
Type_1_Dark,0.466,0.997,0.997,0.997,0.997,0.997,0.997,0.997,0.997,0.997
Type_1_Dragon,0.649,0.987,0.998,0.998,0.997,0.998,0.998,0.998,0.998,0.998
Type_1_Electric,0.161,0.995,0.185,0.994,0.994,0.995,0.994,0.995,0.997,0.994
Type_1_Fairy,0.971,0.999,0.999,0.999,0.999,0.999,0.999,0.999,0.999,0.999
Type_1_Fighting,0.249,0.999,0.999,0.999,0.999,0.999,0.999,0.997,0.999,0.999
Type_1_Fire,0.058,0.997,0.058,0.997,0.997,0.997,0.997,0.995,0.997,0.997
Type_1_Flying,0.996,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
Type_1_Ghost,0.158,0.999,0.479,1.000,1.000,1.000,1.000,1.000,1.000,1.000
Type_1_Grass,0.056,0.995,0.043,0.995,0.996,0.995,0.996,0.993,0.996,0.996


In [168]:
accuracy_w.describe().round(3)

,Naive Bayes,LDA,QDA,Logistic Regression,Decision Tree,Random Forest,Extremely Random Forest,Adaptive Boosting,Gradient Boosting,Final Ensemble
count,18.000,18.000,18.000,18.000,18.000,18.000,18.000,18.000,18.000,18.000
mean,0.325,0.995,0.626,0.995,0.995,0.995,0.995,0.994,0.996,0.996
std,0.293,0.006,0.436,0.006,0.006,0.006,0.006,0.008,0.006,0.006
min,0.052,0.978,0.031,0.978,0.978,0.979,0.978,0.965,0.973,0.978
25%,0.130,0.995,0.155,0.995,0.995,0.995,0.995,0.994,0.996,0.996
50%,0.205,0.997,0.989,0.997,0.997,0.997,0.997,0.997,0.997,0.997
75%,0.454,0.998,0.998,0.999,0.999,0.999,0.999,0.998,0.998,0.999
max,0.996,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
